In [ ]:
import yaml
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from urllib.parse import quote

In [ ]:
with open('./config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_oltp= config['AW_OLTP']
    config_olap= config['AW_OLAP']
    print(config_oltp)

In [ ]:
url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{quote(config_oltp['password'])}@{config_oltp['host']}:"
            f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{quote(config_olap['password'])}@{config_olap['host']}:"
            f"{config_olap['port']}/{config_olap['dbname']}?sslmode=require")

print(url_oltp)
print(url_olap)

In [ ]:
oltp = create_engine(url_oltp)
olap = create_engine(url_olap)

In [ ]:
inspector_oltp = inspect(oltp)
inspector_olap = inspect(olap)

In [ ]:
dim_product = pd.read_sql_table('DimCurrency', oltp, schema='public')
dim_date = pd.read_sql_table('DimDate', oltp, schema='public')
dim_promotion = pd.read_sql_table('DimPromotion', oltp, schema='public')
dim_sales_reason = pd.read_sql_table('DimSalesTerritory', oltp, schema='public')

In [ ]:
fact_internet_sales = pd.read_sql_table('SalesOrderDetail', oltp, schema='Sales')
sales_order_header = pd.read_sql_table('SalesOrderHeader', oltp, schema='Sales', columns=['SalesOrderID', 'OrderDate', 'DueDate', 'ShipDate', 'ShipToAddressID', 'BillToAddressID', 'ShipMethodID', 'CreditCardID', 'CurrencyRateID', 'SalesTerritoryID', 'SalesPersonID', 'CustomerID', 'TotalDue'])


In [ ]:
fact_internet_sales = fact_internet_sales.merge(sales_order_header, on='SalesOrderID', how='left')
fact_internet_sales = fact_internet_sales.merge(dim_product[['ProductAlternativeKey', 'EnglishProductName']], left_on='ProductID', right_index=True, how='left')
fact_internet_sales = fact_internet_sales.merge(dim_customer[['CustomerFirstName', 'CustomerLastName']], left_on='CustomerID', right_index=True, how='left')
fact_internet_sales = fact_internet_sales.merge(dim_currency[['CurrencyAlternateKey']], left_on='CurrencyRateID', right_index=True, how='left')
fact_internet_sales = fact_internet_sales.merge(dim_promotion[['PromotionKey']], left_on='SpecialOfferID', right_index=True, how='left')
fact_internet_sales = fact_internet_sales.merge(dim_sales_territory[['SalesTerritoryKey']], left_on='SalesTerritoryID', right_index=True, how='left')


In [ ]:
fact_internet_sales = fact_internet_sales.rename(columns={
    'SalesOrderID': 'InternetSalesOrderKey',
    'OrderDate': 'OrderDate',
    'DueDate': 'DueDate',
    'ShipDate': 'ShipDate',
    'ProductAlternativeKey': 'ProductKey',
    'CustomerID': 'CustomerKey',
    'CurrencyRateID': 'CurrencyKey',
    'SpecialOfferID': 'PromotionKey',
    'SalesTerritoryID': 'SalesTerritoryKey',
    'TotalDue': 'SalesAmount'
})

In [ ]:
fact_internet_sales = fact_internet_sales.drop(columns=['rowguid', 'ModifiedDate', 'ProductID'])

In [ ]:
fact_internet_sales.head()